In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [2]:
import pandas as pd
import numpy as np 

In [4]:
train = c.data.make_base_df()
train = c.processing.transform_save_imgs(train, c.processing.resize_100
train = c.processing.random_forest_transform(train, 'resize_100_path', grayscale=False)

In [ ]:
test = c.data.make_test_df()
test = c.processing.transform_save_imgs(test, c.processing.resize_100)
test = c.processing.random_forest_transform(test, 'resize_100_path', grayscale=False)

In [6]:
from sklearn.ensemble import RandomForestClassifier as RFC
forest = RFC(n_jobs=2,n_estimators=50)

np.squeeze(train['vec'].as_matrix())

#Bit of a nasty hack to get the vectors into the right form - a (888,10000) numpy array
train_vecs = np.squeeze(np.array(train['vec'].tolist()))
forest.fit(train_vecs, train['Type'].as_matrix())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [12]:
test_vecs = np.squeeze(np.array(test['vec'].tolist()))

preds_probs = forest.predict_proba(test_vecs)

In [9]:
test = c.processing.append_probabilities(test, preds_probs, forest.classes_)

In [10]:
test.head()

,path,type,filetype,num_id,grayscale_resize_path,vec,Type_1,Type_2,Type_3
1067,../data/train/Type_3/1107.jpg,3,jpg,1107,../data/processed/grayscale_resize/train/1107.jpg,"[[0.0158690181357, 0.0158690181357, 0.01586901...",0.08,0.66,0.26
1305,../data/train/Type_3/49.jpg,3,jpg,49,../data/processed/grayscale_resize/train/49.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.30,0.32,0.38
328,../data/train/Type_2/1138.jpg,2,jpg,1138,../data/processed/grayscale_resize/train/1138.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0131317753844, 0.01313...",0.20,0.52,0.28
704,../data/train/Type_2/428.jpg,2,jpg,428,../data/processed/grayscale_resize/train/428.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.12,0.42,0.46
191,../data/train/Type_1/7.jpg,1,jpg,7,../data/processed/grayscale_resize/train/7.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.34,0.30,0.36


In [9]:
c.submission.write_submission_file('output2.csv', test)